# LLM Iteration 5

**Aim**

This has the same aim as iteration 4, but the initial brief given to the LLM in iteration 3 will be provided in order to for LLM to understand the model better. The update should mean a random patient type is not admitted each time, but different patient types arrive independently of each other. It should also be able to input a different random seed for each patient type for reproducibility and testing.

**Prompt**

The code below has been designed on the following brief:

Code a discrete-event simulation model of a stroke ward in a hospital in python 3.10 and simpy 4.
Please code the full model specified. Do not return a simplified version. Show all code. 

All time units in the model are in days.
Each patient in the model has a unique identifier. The first patient to arrive has an identifier of 0. For each subsequent patient increment the identifier by 1.

Patients to arrive in a stroke ward that admits four different patient types: Stroke, TIA, Complex Neurological or Other. Each patient type had a different interarrival time distribution and each patient type arrival is independent of the other patient types. There are 10 beds on the stroke ward. The patient should only be admitted if there is an available bed. The patient holds the bed until their length of stay has elapsed. The length of stay for all patients is 7 days. After the length of stay has elapsed, the patient leaves the model. 

All patient types must have their own generator function. 

The model should print out useful information after each event. 
The model should include a user settable run length. This defaults to 2 months. 

The interarrival time distribution and parameters of patients are dependent on the patient type. This describes the average time between required admissions. For each distribution time is measured in days. 
Stroke = Exponential: 1.2
TIA = Exponential: 9.3
Complex Neurological = Exponential: 3.6
Other = Exponential: 3.2

Numpy should be used for sampling. Each interarrival distribution should have its own numpy.random.Generator object. 

```python
import simpy
import numpy as np

NUM_BEDS = 10
LENGTH_OF_STAY = 7
RUN_LENGTH = 60  # Default run length in days (2 months)
INTERARRIVAL_TIMES = {
    'Stroke': 1.2,
    'TIA': 9.3,
    'Complex Neurological': 3.6,
    'Other': 3.2
}

class StrokeWard:
    def __init__(self, env):
        self.env = env
        self.beds = simpy.Resource(env, capacity=NUM_BEDS)
        self.patient_count = 0

    def admit_patient(self, patient):
        with self.beds.request() as bed:
            yield bed
            print(f"Patient {patient.id} of type {patient.type} admitted at day {round(self.env.now, 2)}")
            yield self.env.timeout(LENGTH_OF_STAY)
            print(f"Patient {patient.id} of type {patient.type} discharged at day {round(self.env.now, 2)}")

class Patient:
    def __init__(self, id, type):
        self.id = id
        self.type = type

def patient_generator(env, ward, interarrival_time, random_seed=None):
    rng = np.random.default_rng(random_seed)
    while True:
        yield env.timeout(rng.exponential(interarrival_time))
        patient_type = np.random.choice(list(INTERARRIVAL_TIMES.keys()))
        patient = Patient(ward.patient_count, patient_type)
        ward.patient_count += 1
        env.process(ward.admit_patient(patient))

env = simpy.Environment()
ward = StrokeWard(env)
env.process(patient_generator(env, ward, INTERARRIVAL_TIMES['Stroke'], random_seed=42))
env.run(until=RUN_LENGTH)
```

Update this code to use the correct interarrival time for each patient type and have the option to input a different random seed for each patient type.

**Note**

Modifications have been made to the output from the LLM, indicated by:
```python
########## MODIFICATION ##########
change
##################################
```
This is either used to add in code from previous iterations if only certain sections were updated, used to correct the LLM's code if required, or used to check the model is working as expected.

**LLM Output**

In [1]:
import simpy
import numpy as np

In [2]:
# Define the parameters
NUM_BEDS = 10
LENGTH_OF_STAY = 7
RUN_LENGTH = 60  # Default run length in days (2 months)
INTERARRIVAL_TIMES = {
    'Stroke': 1.2,
    'TIA': 9.3,
    'Complex NL': 3.6,
    'Other': 3.2
}

In [3]:
class StrokeWard:
    def __init__(self, env):
        self.env = env
        self.beds = simpy.Resource(env, capacity=NUM_BEDS)
        self.patient_count = 0
        ########## MODIFICATION ##########
        self.patients_in_beds = 0
        self.bed_in_use_store = []
        self.ward_admit_time = []
        self.los_store = []
        ##################################

    def admit_patient(self, patient):
        with self.beds.request() as bed:
            yield bed
            ########## MODIFICATION ##########
            bed_arrival_time = self.env.now
            self.ward_admit_time.append(bed_arrival_time)
            self.patients_in_beds += 1
            self.bed_in_use_store.append(self.patients_in_beds)
            ##################################
            print(f'Patient {patient.id} of type {patient.type} admitted at '
                  f'day {round(self.env.now, 2)}')
            yield self.env.timeout(LENGTH_OF_STAY)

            ########## MODIFICATION ##########
            self.patients_in_beds -= 1
            bed_leave_time = self.env.now
            self.los_store.append(bed_leave_time - bed_arrival_time)
            ##################################
            print(f'Patient {patient.id} of type {patient.type} discharged at '
                  f'day {round(self.env.now, 2)}')

class Patient:
    def __init__(self, id, type):
        self.id = id
        self.type = type

def patient_generator(env, ward, interarrival_time, patient_type,
                      random_seed=None):
    rng = np.random.default_rng(random_seed)
    while True:
        yield env.timeout(rng.exponential(interarrival_time))
        patient = Patient(ward.patient_count, patient_type)
        ward.patient_count += 1
        env.process(ward.admit_patient(patient))

In [4]:
# Setup and start the simulation
env = simpy.Environment()
ward = StrokeWard(env)

# Initialize patient generators for each patient type with different 
# interarrival times and random seeds
env.process(patient_generator(env, ward, INTERARRIVAL_TIMES['Stroke'],
                              'Stroke', random_seed=42))
env.process(patient_generator(env, ward, INTERARRIVAL_TIMES['TIA'], 'TIA',
                              random_seed=10))
env.process(patient_generator(env, ward, INTERARRIVAL_TIMES['Complex NL'],
                              'Complex NL', random_seed=20))
env.process(patient_generator(env, ward, INTERARRIVAL_TIMES['Other'], 'Other',
                              random_seed=30))

env.run(until=RUN_LENGTH)

Patient 0 of type Complex NL admitted at day 0.77
Patient 1 of type Other admitted at day 2.41
Patient 2 of type Complex NL admitted at day 2.69
Patient 3 of type Stroke admitted at day 2.89
Patient 4 of type Other admitted at day 3.0
Patient 5 of type Complex NL admitted at day 3.17
Patient 6 of type Other admitted at day 4.22
Patient 7 of type Complex NL admitted at day 5.34
Patient 8 of type Other admitted at day 5.35
Patient 9 of type Stroke admitted at day 5.69
Patient 0 of type Complex NL discharged at day 7.77
Patient 10 of type Complex NL admitted at day 7.77
Patient 1 of type Other discharged at day 9.41
Patient 11 of type Complex NL admitted at day 9.41
Patient 2 of type Complex NL discharged at day 9.69
Patient 12 of type Complex NL admitted at day 9.69
Patient 3 of type Stroke discharged at day 9.89
Patient 13 of type TIA admitted at day 9.89
Patient 4 of type Other discharged at day 10.0
Patient 14 of type Stroke admitted at day 10.0
Patient 5 of type Complex NL discharged

**Checks**

In [5]:
class TestCase:
    def __init__(self, env, ward):
        self.env = env
        self.ward = ward
        self.env.process(patient_generator(self.env, self.ward,
                                           INTERARRIVAL_TIMES['Stroke'],
                                           'Stroke', random_seed=42))
        self.env.process(patient_generator(self.env, self.ward,
                                           INTERARRIVAL_TIMES['TIA'], 'TIA',
                                           random_seed=10))
        self.env.process(patient_generator(self.env, self.ward,
                                           INTERARRIVAL_TIMES['Complex NL'],
                                           'Complex NL', random_seed=20))
        self.env.process(patient_generator(self.env, self.ward,
                                           INTERARRIVAL_TIMES['Other'],
                                           'Other', random_seed=30))

    def test_run_length(self):
        '''
        Checks that specified run length is implemented correctly.
        '''
        self.env.run(until=RUN_LENGTH)

        if self.env.now > RUN_LENGTH:
            print('The simulation has continued for longer than input.')

    def test_bed_hold(self):
        '''
        Checks that patients are holding beds until they leave the model.
        '''
        self.env.run(until=RUN_LENGTH)

        for bed_use in ward.bed_in_use_store:
            if bed_use > NUM_BEDS:
                print('The number of beds in use exceeded limit.')

    def test_length_of_stay(self):
        '''
        Checks that specified length of stay correctly implemented.
        '''
        self.env.run(until=RUN_LENGTH)

        for los in ward.los_store:
            if los > LENGTH_OF_STAY:
                print('Patient length of stay exceeded limit.')
            elif los < LENGTH_OF_STAY:
                print('Patient did not stay for the full duration.')

In [6]:
# Checks run length
NUM_BEDS = 10
LENGTH_OF_STAY = 7
RUN_LENGTH = 10

env = simpy.Environment()
ward = StrokeWard(env)

test = TestCase(env, ward)
test.test_run_length()

Patient 0 of type Complex NL admitted at day 0.77
Patient 1 of type Other admitted at day 2.41
Patient 2 of type Complex NL admitted at day 2.69
Patient 3 of type Stroke admitted at day 2.89
Patient 4 of type Other admitted at day 3.0
Patient 5 of type Complex NL admitted at day 3.17
Patient 6 of type Other admitted at day 4.22
Patient 7 of type Complex NL admitted at day 5.34
Patient 8 of type Other admitted at day 5.35
Patient 9 of type Stroke admitted at day 5.69
Patient 0 of type Complex NL discharged at day 7.77
Patient 10 of type Complex NL admitted at day 7.77
Patient 1 of type Other discharged at day 9.41
Patient 11 of type Complex NL admitted at day 9.41
Patient 2 of type Complex NL discharged at day 9.69
Patient 12 of type Complex NL admitted at day 9.69
Patient 3 of type Stroke discharged at day 9.89
Patient 13 of type TIA admitted at day 9.89
Patient 4 of type Other discharged at day 10.0
Patient 14 of type Stroke admitted at day 10.0


In [7]:
# Checks that is only ever 1 patient if there is only 1 bed
NUM_BEDS = 1
LENGTH_OF_STAY = 7
RUN_LENGTH = 60

env = simpy.Environment()
ward = StrokeWard(env)

test = TestCase(env, ward)
test.test_bed_hold()

Patient 0 of type Complex NL admitted at day 0.77
Patient 0 of type Complex NL discharged at day 7.77
Patient 1 of type Other admitted at day 7.77
Patient 1 of type Other discharged at day 14.77
Patient 2 of type Complex NL admitted at day 14.77
Patient 2 of type Complex NL discharged at day 21.77
Patient 3 of type Stroke admitted at day 21.77
Patient 3 of type Stroke discharged at day 28.77
Patient 4 of type Other admitted at day 28.77
Patient 4 of type Other discharged at day 35.77
Patient 5 of type Complex NL admitted at day 35.77
Patient 5 of type Complex NL discharged at day 42.77
Patient 6 of type Other admitted at day 42.77
Patient 6 of type Other discharged at day 49.77
Patient 7 of type Complex NL admitted at day 49.77
Patient 7 of type Complex NL discharged at day 56.77
Patient 8 of type Other admitted at day 56.77


In [8]:
# Checks if stay length correctly implemented
NUM_BEDS = 10
LENGTH_OF_STAY = 50
RUN_LENGTH = 60

env = simpy.Environment()
ward = StrokeWard(env)

test = TestCase(env, ward)
test.test_length_of_stay()

Patient 0 of type Complex NL admitted at day 0.77
Patient 1 of type Other admitted at day 2.41
Patient 2 of type Complex NL admitted at day 2.69
Patient 3 of type Stroke admitted at day 2.89
Patient 4 of type Other admitted at day 3.0
Patient 5 of type Complex NL admitted at day 3.17
Patient 6 of type Other admitted at day 4.22
Patient 7 of type Complex NL admitted at day 5.34
Patient 8 of type Other admitted at day 5.35
Patient 9 of type Stroke admitted at day 5.69
Patient 0 of type Complex NL discharged at day 50.77
Patient 10 of type Complex NL admitted at day 50.77
Patient 1 of type Other discharged at day 52.41
Patient 11 of type Complex NL admitted at day 52.41
Patient 2 of type Complex NL discharged at day 52.69
Patient 12 of type Complex NL admitted at day 52.69
Patient 3 of type Stroke discharged at day 52.89
Patient 13 of type TIA admitted at day 52.89
Patient 4 of type Other discharged at day 53.0
Patient 14 of type Stroke admitted at day 53.0
Patient 5 of type Complex NL di

In [9]:
# Test random seeds output the same result
NUM_BEDS = 10
LENGTH_OF_STAY = 7
RUN_LENGTH = 15

env = simpy.Environment()
ward1 = StrokeWard(env)
env.process(patient_generator(env, ward1, INTERARRIVAL_TIMES['Stroke'],
                              'Stroke', random_seed=1))
env.process(patient_generator(env, ward1, INTERARRIVAL_TIMES['TIA'], 'TIA',
                              random_seed=2))
env.process(patient_generator(env, ward1, INTERARRIVAL_TIMES['Complex NL'],
                              'Complex NL', random_seed=3))
env.process(patient_generator(env, ward1, INTERARRIVAL_TIMES['Other'], 'Other',
                              random_seed=4))
env.run(until=RUN_LENGTH)
print('\n')

env = simpy.Environment()
ward2 = StrokeWard(env)
env.process(patient_generator(env, ward2, INTERARRIVAL_TIMES['Stroke'],
                              'Stroke', random_seed=1))
env.process(patient_generator(env, ward2, INTERARRIVAL_TIMES['TIA'], 'TIA',
                              random_seed=2))
env.process(patient_generator(env, ward2, INTERARRIVAL_TIMES['Complex NL'],
                              'Complex NL', random_seed=3))
env.process(patient_generator(env, ward2, INTERARRIVAL_TIMES['Other'], 'Other',
                              random_seed=4))
env.run(until=RUN_LENGTH)

for arrival1, arrival2 in zip(ward1.ward_admit_time, ward2.ward_admit_time):
    if arrival1 != arrival2:
        print('Different arrival times on the ward for the same random seed.')

Patient 0 of type Complex NL admitted at day 0.4
Patient 1 of type TIA admitted at day 1.21
Patient 2 of type Stroke admitted at day 1.29
Patient 3 of type Stroke admitted at day 1.66
Patient 4 of type Complex NL admitted at day 1.8
Patient 5 of type TIA admitted at day 3.24
Patient 6 of type Complex NL admitted at day 6.84
Patient 0 of type Complex NL discharged at day 7.4
Patient 7 of type TIA admitted at day 8.02
Patient 8 of type Stroke admitted at day 8.11
Patient 1 of type TIA discharged at day 8.21
Patient 2 of type Stroke discharged at day 8.29
Patient 9 of type Stroke admitted at day 8.55
Patient 3 of type Stroke discharged at day 8.66
Patient 10 of type Stroke admitted at day 8.69
Patient 4 of type Complex NL discharged at day 8.8
Patient 5 of type TIA discharged at day 10.24
Patient 11 of type Stroke admitted at day 10.85
Patient 12 of type Stroke admitted at day 11.44
Patient 13 of type Stroke admitted at day 12.11
Patient 14 of type Stroke admitted at day 12.14
Patient 15 

**Reflection**

Patients of all types are now being admitted to the stroke ward independently of each other as a new process has been set up for each patient type. Random seeds can be set for each patient type, meaning the results are repeatable as the same type of patient is output each time the model is run. Patients are also holding the beds until they leave so no more patients are being admitted. The length of stay and run time are also working correctly as before.